In [ ]:
import polars as pl
from datetime import datetime
import get_and_store as gas
import polars_indicators as pi
from polars_indicators import IndicatorResult

In [ ]:
symbols = ['A','AA']

In [ ]:
df = gas.get_symbols_df(symbols)
df = df.filter(pl.col('Date') >= datetime(2023, 1, 1))
df = df.collect()

In [ ]:
df

In [ ]:
def add_false_column_with_true_indicies(df, indicies: list[int], column: str):
    index = "index"
    new_columns = df.columns.copy()
    new_columns.append(column)
    return df.with_row_count(name=index).with_columns( 
            pl.when(
                pl.col(index).is_in(indicies)) \
                .then(pl.lit(True)) \
            .otherwise(
                pl.lit(False)) \
            .alias(column)).select(new_columns) 
add_false_column_with_true_indicies(df, [2, 3, 7], "mycol")

In [ ]:
index_name = 'my_index'
enter_column = "enter"
mydf = add_false_column_with_true_indicies(df, [3, 4, 7, 85], enter_column).with_row_count(index_name)

mydf

In [ ]:
trail = pi.trailing_stop(mydf, 2)
exit_column = trail.column

In [ ]:
pi.create_trade_ids(trail.df, enter_column, exit_column).df